In [3]:
import gensim
import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
#nltk.download()
import string 
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import pickle 
import pandas as pd 
import os

print("out")

#case metadata
casedata = pd.read_pickle('casedata.pkl')

#taking only year 2013
yearr="2013"
df=casedata[(casedata["year"] == yearr)]
print((df).head(3))

#reading documents
yearr="2013"
data=[]
docLabels=[]
for i in range(1,13):
    pathname = os.path.join("Filtered/"+yearr, str(i))
    files=os.listdir(pathname)
    
    for file in files:
        file1=file.split("_")[0]
        caseid=file1
        if caseid not in docLabels:
            
            try:
                data.append(open(pathname+"/"+file).read())
            except:
                print(file)
                print(i)
                continue
            docLabels.append(caseid)


        else:
            print(file)
            print(i)
print(len(docLabels))
print(len(data))

out
               year     date month  category      sub-category  \
caseid                                                           
X10OE4DGG000N  2013  7/16/13     7       6.0    Employment Law   
X1B7QSOG0000N  2013  8/15/13     8       6.0         Labor Law   
X1ERHAGN       2013   1/9/13     1       7.0  Tax & Accounting   

                                                     filename quarter  
caseid                                                                 
X10OE4DGG000N         X10OE4DGG000N_contentMajOp_HAMILTON.txt       3  
X1B7QSOG0000N  X1B7QSOG0000N_contentMajOp_BOYCE F. MARTIN.txt       3  
X1ERHAGN                        X1ERHAGN_contentMajOp_DYK.txt       1  
.DS_Store
1
X1JR2IQ003_contentMajOp_E. GRADY JOLLY.txt
6
X1K7E4U003_contentMajOp_PLAGE R.txt
8
X1K89GO003_contentMajOp_LOURIE.txt
8
481
481


In [4]:



def clean(text):

    text=text.lower()
    table = str.maketrans("", "", string.punctuation)
    wnl = nltk.WordNetLemmatizer()
    
    
    text=text.translate(table)
    #print("without punctuation ")
    #print(text)
    token = nltk.word_tokenize(text)
    #print("tokenized")
    #print(token)
    

    stop_words = set(stopwords.words('english'))
    filtered_sentence = []

    for w in token:
        if w not in stop_words:
            filtered_sentence.append(w)
   
    token=filtered_sentence
    #print("without stop words")
    #print(token)
    token=[wnl.lemmatize(t) for t in token]
    #print("after lemma")
    #print(token)
    return token

new_data=[]
for d in data:
    d=clean(d)
    new_data.append(d)
data=new_data

In [5]:
#create iterator of data and doclabels
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
              yield gensim.models.doc2vec.LabeledSentence(doc,[self.labels_list[idx]])


In [6]:

it = LabeledLineSentence(data, docLabels)


In [7]:
model = gensim.models.Doc2Vec(size=300, min_count=0, alpha=0.025, min_alpha=0.025)
model.build_vocab(it)

#training 
for epoch in range(100):
    #print('iteration '+str(epoch+1))
    model.train(it,total_examples=model.corpus_count,epochs=model.epochs)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

model.save('doc2vec.model')
print("model saved")

/Users/amaladeshpande/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/Users/amaladeshpande/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  


model saved


In [8]:
d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec.model')

In [9]:
columns1=['CaseId','Date','Vector','Year','Month']

i=0
docvector= pd.DataFrame(columns=columns1)
for caseid in docLabels:
    
    date=df.loc[caseid]["date"]
    m=df.loc[caseid]["month"]
    m=int(m)
    y=df.loc[caseid]["year"]
    docvec = d2v_model.docvecs[caseid]
    docvector = docvector.append({'CaseId':caseid,'Date':date,'Vector':docvec,'Month':m,'Year':y},ignore_index=True)
docvector.head(10)

,CaseId,Date,Vector,Year,Month
0,X1F0UMON,1/10/13,"[-0.702887, -2.72339, 0.152043, -1.59495, -2.2...",2013,1
1,X1J3RP6003,1/14/13,"[3.23643, -0.0937236, 4.61111, 1.98329, -2.810...",2013,1
2,X1J2U46003,1/4/13,"[-1.2397, -0.367636, -2.35951, -2.98144, -2.83...",2013,1
3,X1J62O6003,1/28/13,"[0.83, -0.859892, 1.11428, 1.27963, 0.127096, ...",2013,1
4,X1EU3KGN,1/17/13,"[-3.73752, -0.2601, -1.1849, -2.61639, -3.7697...",2013,1
5,X1J2NRO003,1/2/13,"[-0.331647, -2.47936, 2.81137, 3.15342, -3.732...",2013,1
6,X1Q6MITDN5O2,1/28/13,"[0.478112, -1.36496, -0.223039, 1.25757, -2.78...",2013,1
7,X1J3JV8003,1/11/13,"[0.525288, 2.7498, 1.56802, -2.44837, -2.05702...",2013,1
8,X1J3FQQ003,1/10/13,"[0.794932, 3.1061, -0.873964, 1.08231, -2.2346...",2013,1
9,X1J6PGC003,1/31/13,"[0.79056, 3.62492, 3.95146, 0.386374, -2.00166...",2013,1


In [10]:
#finding similar docs
sims = d2v_model.docvecs.most_similar('XVITNB80000N')
print((sims))

[('X1GDMUON', 0.49876973032951355), ('X1JE6MG003', 0.4984654486179352), ('X1KU5HS003', 0.49731796979904175), ('X1GFHF8N', 0.4941115379333496), ('X1K9LH0003', 0.4900771975517273), ('X1J434M003', 0.4872015714645386), ('X1GBNA0N', 0.486237496137619), ('X1JLVJC003', 0.4849395453929901), ('X1GFE5ON', 0.48279041051864624), ('X1ESKFON', 0.4826751947402954)]


In [11]:
docvector.to_pickle("docvector.pkl")

In [12]:
import pickle 
with open('docvector.pkl', 'rb') as f:
    docvec = pickle.load(f)
docvec.head(3)

,CaseId,Date,Vector,Year,Month
0,X1F0UMON,1/10/13,"[-0.702887, -2.72339, 0.152043, -1.59495, -2.2...",2013,1
1,X1J3RP6003,1/14/13,"[3.23643, -0.0937236, 4.61111, 1.98329, -2.810...",2013,1
2,X1J2U46003,1/4/13,"[-1.2397, -0.367636, -2.35951, -2.98144, -2.83...",2013,1


In [13]:
train=docvec[docvec["Month"]<=7]
print(len(train))      
train.head(3)

337


,CaseId,Date,Vector,Year,Month
0,X1F0UMON,1/10/13,"[-0.702887, -2.72339, 0.152043, -1.59495, -2.2...",2013,1
1,X1J3RP6003,1/14/13,"[3.23643, -0.0937236, 4.61111, 1.98329, -2.810...",2013,1
2,X1J2U46003,1/4/13,"[-1.2397, -0.367636, -2.35951, -2.98144, -2.83...",2013,1


In [14]:
test=docvec[docvec["Month"]>7]
print(len(test)) 
test.head(3)

144


,CaseId,Date,Vector,Year,Month
337,X1K9LH0003,8/26/13,"[-1.56573, 0.42477, -0.0676439, -1.27875, -0.9...",2013,8
338,X1K8D5O003,8/15/13,"[-3.31128, -0.201175, 2.06844, 4.52087, -2.472...",2013,8
339,X1K7LQ2003,8/8/13,"[2.93222, 3.85807, 1.08474, 0.228436, 0.120343...",2013,8


In [15]:
train.to_pickle("traindocvector.pkl")
test.to_pickle("testdocvector.pkl")